In [12]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
#Find path of each data files
eng_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ENG-League-2016-2020.csv'
esp_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ESP-League-2016-2020.csv'
fra_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/FRA-League-2016-2020.csv'
ger_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/GER-League-2016-2020.csv'
ita_path = '/Users/selbl/Desktop/PhD/Second Year/First Quarter/CS 230/Project/Data/ITA-League-2016-2020.csv'
#Read each dataframe
df_eng = pd.read_csv(eng_path)
df_esp = pd.read_csv(esp_path)
df_fra = pd.read_csv(fra_path)
df_ger = pd.read_csv(ger_path)
df_ita = pd.read_csv(ita_path)
#Combine
df = [df_eng, df_esp, df_fra,df_ger,df_ita]
df = pd.concat(df)

In [10]:
#Get distribution of results
df['Result'].value_counts()

W    3294
L    2149
D    1790
Name: Result, dtype: int64

In [13]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1/(1 + np.exp(-z))
    
    return s

In [14]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias) of type float
    """
    w = np.zeros(dim).reshape(dim,1)
    b = 0.0

    return w, b

In [15]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    #Forward Prop
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)
    cost = -(1/m)*(np.dot(Y,np.log(A).T) + np.dot(1 - Y,np.log(1 - A).T))
    #Backward Prop
    dw = (1/m)*np.dot(X,(A - Y).T)
    db = (1/m)*(np.dot((A - Y),np.ones(m)))[0]
    cost = np.squeeze(np.array(cost))
    #Store
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [16]:
def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        w = w - learning_rate*dw
        b = b - learning_rate*db
        # YOUR CODE ENDS HERE
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
            # Print the cost every 100 training iterations
            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [29]:
#Initialize parameters
dim = 2
w, b = initialize_with_zeros(dim)
X = df[['Home_Elo','Away_Elo']].T
Y = pd.Categorical(df['Result']).codes
#L is 1, D is 0, W is 2
params, grads, costs = optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print("Costs = " + str(costs))


w = [[338.58927377]
 [310.0385983 ]]
b = 0.19164226462048978
dw = [[-367.82074036]
 [-336.09705182]]
db = -0.20793584957832156
Costs = [array(0.69314718)]


<ipython-input-15-cebcd6ef516c>:22: RuntimeWarning: divide by zero encountered in log
  cost = -(1/m)*(np.dot(Y,np.log(A).T) + np.dot(1 - Y,np.log(1 - A).T))


In [30]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T,X) + b)
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[0,i] > 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0    
    return Y_prediction

In [31]:
w = np.array([[0.1124579], [0.23106775]])
b = -0.3
X = np.array([[1., -1.1, -3.2],[1.2, 2., 0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[1. 1. 0.]]
